In [1]:
import gzip
import json
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.model_selection import train_test_split
import torch
import torch_geometric
# import tensorflow as tf
import scipy.sparse as sp

import math
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.optim as optim

import time
import argparse

In [2]:
with gzip.open('NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))

In [3]:
instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

conn=np.load('NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)

In [4]:
A.toarray()

array([[21,  1,  1, ...,  0,  1,  2],
       [ 1, 21,  1, ...,  0,  1,  2],
       [ 1,  1, 21, ...,  0,  1,  2],
       ...,
       [ 0,  0,  0, ...,  5,  0,  0],
       [ 1,  1,  1, ...,  0,  5,  2],
       [ 2,  2,  2, ...,  0,  2,  5]])

In [5]:
def buildBST(array,start=0,finish=-1):
    if finish<0:
        finish = len(array)
    mid = (start + finish) // 2
    if mid-start==1:
        ltl=start
    else:
        ltl=buildBST(array,start,mid)
    
    if finish-mid==1:
        gtl=mid
    else:
        gtl=buildBST(array,mid,finish)
        
    return((array[mid],ltl,gtl))

In [6]:
congestion_data = np.load('NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar_congestion.npz')
xbst=buildBST(congestion_data['xBoundaryList'])
ybst=buildBST(congestion_data['yBoundaryList'])
demand = np.zeros(shape = [instances.shape[0],])


In [7]:
def getGRCIndex(x,y,xbst,ybst):
    while (type(xbst)==tuple):
        if x < xbst[0]:
            xbst=xbst[1]
        else:
            xbst=xbst[2]
            
    while (type(ybst)==tuple):
        if y < ybst[0]:
            ybst=ybst[1]
        else:
            ybst=ybst[2]
            
    return ybst, xbst

In [8]:
for k in range(instances.shape[0]):
#     print(k)
    xloc = instances.iloc[k]['xloc']; yloc = instances.iloc[k]['yloc']
    i,j=getGRCIndex(xloc,yloc,xbst,ybst)
    d = 0 
    for l in list(congestion_data['layerList']): 
        lyr=list(congestion_data['layerList']).index(l)
        d += congestion_data['demand'][lyr][i][j]
    demand[k] = d
        
instances['routing_demand'] = demand

In [9]:
# Set random seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [10]:

class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [11]:
# class GCNRegression(nn.Module):
#     def __init__(self, nfeat, nhid, dropout):
#         super(GCNRegression, self).__init__()

#         self.gc1 = GraphConvolution(nfeat, nhid)
#         self.gc2 = GraphConvolution(nhid, 1)  # Output layer with 1 unit for regression
#         self.dropout = dropout

#     def forward(self, x, adj):
#         x = F.relu(self.gc1(x, adj))
#         x = F.dropout(x, self.dropout, training=self.training)
#         x = self.gc2(x, adj)
#         return x.squeeze()  # Squeeze the output to have shape (batch_size,)


In [12]:
class GCNRegressionDynamic(nn.Module):
    def __init__(self, nfeat, nhid, dropout, n_layers):
        super(GCNRegressionDynamic, self).__init__()

        self.gc_layers = nn.ModuleList()
        self.dropout = dropout

        # Input layer
        self.gc_layers.append(GraphConvolution(nfeat, nhid))

        # Hidden layers
        for _ in range(n_layers - 2):
            self.gc_layers.append(GraphConvolution(nhid, nhid))

        # Output layer
        self.gc_layers.append(GraphConvolution(nhid, 1))
    
    def forward(self, x, adj):
        for layer in self.gc_layers[:-1]:
            x = F.relu(layer(x, adj))
            x = F.dropout(x, self.dropout, training=self.training)

        # Output layer without ReLU activation
        x = self.gc_layers[-1](x, adj)
        return x.squeeze()


In [13]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [14]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = nn.MSELoss()(output[idx_train], target[idx_train])
    loss_train.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print('Epoch: {:04d}'.format(epoch+1),
                'loss_train: {:.4f}'.format(loss_train.item()),
                'time: {:.4f}s'.format(time.time() - t))
    
def test():
    model.eval()
    output = model(features, adj)
    loss_test = nn.MSELoss()(output[idx_test], target[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()))


In [15]:
instances

,name,id,xloc,yloc,cell,orient,routing_demand
0,clk_gate_out_reg/latch,0,41984,44544,23,0,20.0
1,clk_gate_out_reg_0/latch,1,41984,47616,23,6,23.0
2,clk_gate_out_reg_1/latch,2,44160,44544,23,0,23.0
3,clk_gate_out_reg_2/latch,3,44160,47616,23,0,22.0
4,clk_gate_out_reg_3/latch,4,46336,47616,23,0,21.0
...,...,...,...,...,...,...,...
3947,U4123,3947,21888,53760,42,4,31.0
3948,U4125,3948,33664,66048,42,0,30.0
3949,U4128,3949,23296,66048,34,0,27.0
3950,ZCTSBUF_205_132,3950,40576,44544,11,0,28.0


In [16]:
int(len(instances) * 0.7)

2766

In [17]:
idx_train = range(2766)
idx_test = range(2766, 3952)

idx_train = torch.LongTensor(idx_train)
idx_test = torch.LongTensor(idx_test)

In [18]:
# Instantiate the model
nfeat = 4 # Number of input features
nhid = 64  # Number of hidden units
dropout = 0.2
n_layers = 1
model = GCNRegressionDynamic(nfeat, nhid, dropout, n_layers) #GCNRegression(nfeat, nhid, dropout)

optimizer = optim.Adam(model.parameters(),
                       lr=0.01, weight_decay=5e-4)

# Loss function for regression (Mean Squared Error)
criterion = nn.MSELoss()

In [19]:
features = instances[['xloc', 'yloc', 'cell', 'orient']].to_numpy()
features = sp.csr_matrix(features, dtype=np.float32)
features = normalize(features)
features = torch.FloatTensor(np.array(features.todense()))
# features = torch.tensor(features, dtype=torch.float32)


target = instances[['routing_demand']].to_numpy()
target = torch.tensor(target, dtype=torch.float32).squeeze()

adj = normalize(sp.csr_matrix(A, dtype=np.float32))
adj = torch.tensor(adj.toarray(), dtype=torch.float32) # adjacency matrix

In [20]:
results_df = pd.DataFrame(columns=['Layer', 'Test_MSE', 'Time'])

for i in [1, 2, 3, 4, 5]:
    print("k = {layer}".format(layer=i))
    
    # Modify your model creation and training based on the layer number (i)
    model = GCNRegressionDynamic(nfeat, nhid, dropout, i)
    optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
    criterion = nn.MSELoss()

    t_total = time.time()
    for epoch in range(100):
        train(epoch)
    elapsed_time = time.time() - t_total
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(elapsed_time))

    # Evaluate on the test set
    model.eval()
    output = model(features, adj)
    loss_test = nn.MSELoss()(output[idx_test], target[idx_test])

    # Append the results to the DataFrame
    results_df = results_df.append({'Layer': i, 'Test_MSE': loss_test.item(), 'Time': elapsed_time}, ignore_index=True)


k = 1
Epoch: 0010 loss_train: 553.5036 time: 0.1089s
Epoch: 0020 loss_train: 471.6997 time: 0.1086s
Epoch: 0030 loss_train: 380.0551 time: 0.1087s
Epoch: 0040 loss_train: 279.0865 time: 0.1089s
Epoch: 0050 loss_train: 184.3705 time: 0.1118s
Epoch: 0060 loss_train: 105.7594 time: 0.1088s
Epoch: 0070 loss_train: 55.2399 time: 0.1090s
Epoch: 0080 loss_train: 30.4675 time: 0.1087s
Epoch: 0090 loss_train: 24.3788 time: 0.1096s
Epoch: 0100 loss_train: 24.8498 time: 0.1092s
Optimization Finished!
Total time elapsed: 11.1837s
k = 2
Epoch: 0010 loss_train: 585.0478 time: 0.1103s
Epoch: 0020 loss_train: 498.1251 time: 0.1103s
Epoch: 0030 loss_train: 401.3047 time: 0.1089s
Epoch: 0040 loss_train: 299.0484 time: 0.1093s
Epoch: 0050 loss_train: 199.8082 time: 0.1089s
Epoch: 0060 loss_train: 118.5662 time: 0.1098s
Epoch: 0070 loss_train: 63.4638 time: 0.1107s
Epoch: 0080 loss_train: 34.6123 time: 0.1100s
Epoch: 0090 loss_train: 25.2210 time: 0.1104s
Epoch: 0100 loss_train: 24.1610 time: 0.1092s
Opti

In [21]:
results_df

,Layer,Test_MSE,Time
0,1.0,12.384588,11.183727
1,2.0,12.480188,11.059214
2,3.0,12.794757,22.349082
3,4.0,12.860625,32.920534
4,5.0,12.975776,44.060694


In [22]:
## Addiing an attention layer

In [23]:
class AttentionLayer(Module):
    def __init__(self, in_features, out_features, bias=True):
        super(AttentionLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input):
        att_weights = F.softmax(torch.mm(input, self.weight), dim=1)
        output = att_weights * input
        if self.bias is not None:
            return output + self.bias
        else:
            return output

class GCNAttentionDynamic(nn.Module):
    def __init__(self, nfeat, nhid, dropout, n_layers):
        super(GCNAttentionDynamic, self).__init__()

        self.gc_layers = nn.ModuleList()
        self.att_layers = nn.ModuleList()
        self.dropout = dropout

        # Input layer
        self.gc_layers.append(GraphConvolution(nfeat, nhid))
        self.att_layers.append(AttentionLayer(nhid, nhid))

        # Hidden layers
        for _ in range(n_layers - 2):
            self.gc_layers.append(GraphConvolution(nhid, nhid))
            self.att_layers.append(AttentionLayer(nhid, nhid))

        # Output layer
        self.gc_layers.append(GraphConvolution(nhid, 1))
        self.att_layers.append(AttentionLayer(1, 1))

    def forward(self, x, adj):
        for gc_layer, att_layer in zip(self.gc_layers[:-1], self.att_layers[:-1]):
            x = F.relu(gc_layer(x, adj))
            x = att_layer(x)
            x = F.dropout(x, self.dropout, training=self.training)

        # Output layer without ReLU activation
        x = self.gc_layers[-1](x, adj)
        x = self.att_layers[-1](x)
        return x.squeeze()


In [24]:
# original GCN
model = GCNRegressionDynamic(nfeat, nhid, dropout, 1)
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = nn.MSELoss()

t_total = time.time()
for epoch in range(100):
    train(epoch)
elapsed_time = time.time() - t_total
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(elapsed_time))

# Evaluate on the test set
model.eval()
output = model(features, adj)
loss_test_GCN = nn.MSELoss()(output[idx_test], target[idx_test])


Epoch: 0010 loss_train: 631.8388 time: 0.1279s
Epoch: 0020 loss_train: 558.8183 time: 0.1107s
Epoch: 0030 loss_train: 474.3437 time: 0.1106s
Epoch: 0040 loss_train: 374.0381 time: 0.1108s
Epoch: 0050 loss_train: 269.6749 time: 0.1108s
Epoch: 0060 loss_train: 175.6591 time: 0.1106s
Epoch: 0070 loss_train: 101.1636 time: 0.1113s
Epoch: 0080 loss_train: 53.4536 time: 0.1105s
Epoch: 0090 loss_train: 30.4796 time: 0.1109s
Epoch: 0100 loss_train: 25.1042 time: 0.1106s
Optimization Finished!
Total time elapsed: 11.2215s


In [25]:
# GCN with attention
model = GCNAttentionDynamic(nfeat, nhid, dropout, 1)
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = nn.MSELoss()

t_total = time.time()
for epoch in range(100):
    train(epoch)
elapsed_time = time.time() - t_total
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(elapsed_time))

# Evaluate on the test set
model.eval()
output = model(features, adj)
loss_test_attention = nn.MSELoss()(output[idx_test], target[idx_test])

Epoch: 0010 loss_train: 580.4120 time: 0.1262s
Epoch: 0020 loss_train: 488.3988 time: 0.1152s
Epoch: 0030 loss_train: 394.4220 time: 0.1152s
Epoch: 0040 loss_train: 297.7288 time: 0.1151s
Epoch: 0050 loss_train: 206.7276 time: 0.1162s
Epoch: 0060 loss_train: 129.6535 time: 0.1150s
Epoch: 0070 loss_train: 72.4829 time: 0.1162s
Epoch: 0080 loss_train: 40.5150 time: 0.1151s
Epoch: 0090 loss_train: 26.9751 time: 0.1152s
Epoch: 0100 loss_train: 23.9305 time: 0.1161s
Optimization Finished!
Total time elapsed: 11.7538s


In [26]:
print("GCN Test loss = {GCN}, GCN Test loss with attention layer = {att}".format(GCN = loss_test_GCN, att = loss_test_attention))

GCN Test loss = 13.634328842163086, GCN Test loss with attention layer = 12.835390090942383


# Part 2

In [27]:
# # Initialize an empty adjacency matrix
# adj_matrix = None

# # Initialize an empty adjacency matrix
# total_rows = 0  # You might need to adjust this based on your data
# total_cols = 0  # You might need to adjust this based on your data
# adj_matrix = coo_matrix((total_rows, total_cols), dtype=np.int32)

# # Loop through the files
# for i in range(1, 3):
#     file_path = f'NCSU-DigIC-GraphData-2023-07-25/xbar/{i}/xbar_connectivity.npz'
#     conn = np.load(file_path)

#     # Update the adjacency matrix with the current file's data
#     current_matrix = coo_matrix((conn['data'], (conn['row']+ total_rows, conn['col']+total_cols)), shape=[total_rows, total_cols])
    
#     if adj_matrix is None:
#         adj_matrix = current_matrix
#     else:
#         # Check and adjust the shape if necessary
#         total_rows = max(adj_matrix.shape[0], current_matrix.shape[0])
#         total_cols = max(adj_matrix.shape[1], current_matrix.shape[1])
#         adj_matrix.resize((total_rows, total_cols))
        
#         # Add the current matrix to the adjacency matrix
#         adj_matrix += current_matrix

# # Convert the final adjacency matrix to a dense format if needed
# adj_matrix = adj_matrix.toarray()


In [28]:
# load training files, load xbar 2 as testing
with gzip.open('NCSU-DigIC-GraphData-2023-07-25/xbar/2/xbar.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))

instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

conn=np.load('NCSU-DigIC-GraphData-2023-07-25/xbar/2/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)

congestion_data = np.load('NCSU-DigIC-GraphData-2023-07-25/xbar/2/xbar_congestion.npz')
xbst=buildBST(congestion_data['xBoundaryList'])
ybst=buildBST(congestion_data['yBoundaryList'])
demand = np.zeros(shape = [instances.shape[0],])

for k in range(instances.shape[0]):
#     print(k)
    xloc = instances.iloc[k]['xloc']; yloc = instances.iloc[k]['yloc']
    i,j=getGRCIndex(xloc,yloc,xbst,ybst)
    d = 0 
    for l in list(congestion_data['layerList']): 
        lyr=list(congestion_data['layerList']).index(l)
        d += congestion_data['demand'][lyr][i][j]
    demand[k] = d
        
instances['routing_demand'] = demand

In [29]:
features = instances[['xloc', 'yloc', 'cell', 'orient']].to_numpy()
features = sp.csr_matrix(features, dtype=np.float32)
features = normalize(features)
features = torch.FloatTensor(np.array(features.todense()))
# features = torch.tensor(features, dtype=torch.float32)


target = instances[['routing_demand']].to_numpy()
target = torch.tensor(target, dtype=torch.float32).squeeze()

adj = normalize(sp.csr_matrix(A, dtype=np.float32))
adj = torch.tensor(adj.toarray(), dtype=torch.float32) # adjacency matrix

In [30]:
# Evaluate on the test set
model.eval()
output = model(features, adj)
nn.MSELoss()(output, target)

tensor(188.0422, grad_fn=<MseLossBackward0>)